In [11]:
import pandas as pd

# Load the telecom dataset
# Replace 'path_to_your_dataset.csv' with the actual path to your dataset
data = pd.read_csv('C:/Users/hp/Desktop/New folder (2)/week2/data/Week2_challenge_data_source(CSV).csv')

# Inspect the dataset (optional)
print(data.head())
print(data.columns)

# Assuming the column for handsets is named 'Handset' 
# Change 'Handset' to the actual column name if different
top_handsets = data['Handset Type'].value_counts().head(10)

# Display the top 10 handsets
print("Top 10 Handsets Used by Customers:")
print(top_handsets)

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   